In [40]:
import scipy.io
import numpy as np
import datetime 
import ephem
import random
from scipy import spatial
import matplotlib.pyplot as plt
%matplotlib
from astropy.table import Table
import pandas as pd

data=scipy.io.loadmat('BNS-sources-trial-10saleem.mat')['data'] 
#sourceparameters
DL=data[:,0]
theta=data[:,1]
phi=data[:,2]

Using matplotlib backend: MacOSX


In [2]:
#snr at each detector
snrH=data[:,5]
snrL=data[:,6]
snrV=data[:,7]
snrK=data[:,8]
snrI=data[:,9]
#or for networks of 5 and 3 detectors, one can use directly,
snrLHVKI=np.sqrt(data[:,5]**2 + data[:,6]**2 + data[:,7]**2 +data[:,8]**2 + data[:,9]**2)  
# SNR at LHVKI
snrLHV=np.sqrt(data[:,5]**2 + data[:,6]**2 + data[:,7]**2 )  # LHV*
snrLHVK = np.sqrt(data[:,5]**2 + data[:,6]**2 + data[:,7]**2 + data[:,8]**2)
snrLHVI = np.sqrt(data[:,5]**2 + data[:,6]**2 + data[:,7]**2 + data[:,9]**2)
snrH, snrL, snrLHV,snrLHVK, snrLHVI

(array([1.50828927, 1.3218442 , 4.88369119, ..., 0.14711676, 1.92059118,
        3.73704228]),
 array([2.05597716, 0.7623997 , 3.94531224, ..., 0.87258219, 1.87887723,
        3.17437839]),
 array([2.60109925, 2.01527692, 6.56372816, ..., 2.09722166, 3.03471228,
        5.1190831 ]),
 array([2.9615534 , 2.05214273, 7.23216427, ..., 2.10021926, 3.04157166,
        5.57032074]),
 array([2.80024108, 2.33763304, 7.55768521, ..., 2.6656951 , 3.20978535,
        5.91029335]))

In [4]:
from astropy.time import Time
#import random

def homotime(number):
    """
    Defines homogenously distributed time over the time 2022-2023. Number of time elements is essentially the number of
    theta,  phi you get.(220000)
    Takes time in UTC format and gives out time in UTC format
    """
    time1 = Time('2022-01-01T00:00:00.123456789')
    time1 = time1.jd
    time2 = Time('2023-01-01T00:00:00.123456789')
    time2 = time2.jd
    dt = (time2-time1)/number
    homotime = []
    
    for i in range(number):
        if i%(int((number)/10))==0:print(int(i*100/(number)), '% completed', end='\r')
        timejd = time1 + i*dt
        timeutc = Time(timejd, format='jd')
        homotime.append(timeutc.iso)
    print('***Completed***', end='\r')
    #print(len(homotime))
    return homotime
homotime = homotime(len(theta))


In [7]:
def ecef_eci(lat, lon, time):
    """Convert lat long to right ascention and declination
    lat - latitude of the place where the tigger is at the zenith in degrees
    lon - longitude of the place where the tigger is at the zenith in degrees
    time - UTC time in datetime format
    """
    lat = np.deg2rad(lat)
    lon = np.deg2rad(lon)
    temp = time
    year = int(temp[0:4])
    month = int(temp[5:7])
    day = int(temp[8:10])
    hour = int(temp[11:13])
    minu = int(temp[14:16])
    secs = int(temp[17:19])
    #time = Time(time)
    time = datetime.datetime(year, month,day,hour,minu,secs)
    time0 = datetime.datetime(2000, 1, 1, 12, 0, 0)
    time_sid = 18.697374558 + 24.06570982441908*(time - time0).total_seconds()/86400.0
    time_sid = time_sid - 24.0*np.floor(time_sid/24.0)
    dec = lat
    ra = lon + np.pi/24.0*time_sid
    dec = np.rad2deg(dec)
    ra = np.rad2deg(ra)
    return ra,dec


def eci_ecef(ra, dec, time):
    """Convert ra dec to lattitude and longitude
    ra - right ascention of the body in radians
    dec - declination of the body in radians
    time - UTC time in datetime format
    """
#     ra = np.deg2rad(ra)
#     dec = np.deg2rad(dec)
    temp = time
    year = int(temp[0:4])
    month = int(temp[5:7])
    day = int(temp[8:10])
    hour = int(temp[11:13])
    minu = int(temp[14:16])
    secs = int(temp[17:19])
    #time = Time(time)
    time = datetime.datetime(year, month,day,hour,minu,secs)
    time0 = datetime.datetime(2000, 1, 1, 12, 0, 0)
    time_sid = 18.697374558 + 24.06570982441908*(time - time0).total_seconds()/86400.0
    time_sid = time_sid - 24.0*np.floor(time_sid/24.0)
    lon = (ra - 2*np.pi/24.0*time_sid)
    lat = dec
#     lon = np.rad2deg(lon)
#     lat = np.rad2deg(lat)
    return lat, lon

def is_saa(lat, longi, time):
    """Return whether the satellite is in South Atlantic Anamoly or not.
    Hard bounds used from the data of RXTE and assumed that the SAA region is rectangular below the inclination of RXTE.
    
    ra - right ascention of the body in degrees
    dec - declination of the body in degrees
    time - UTC time in datetime format
    """
    # lat, longi = radec2latlong(ra, dec, time)
    #lat, longi = eci_ecef(ra, dec, time)
    lat = np.rad2deg(lat)
    longi = np.rad2deg(longi)
    if -23< lat <= -7 and -90<=longi<=-10:
        return True
    
#     if -90<=longi<=-10:
#         y = 7*longi/80 + 23/8
#         if lat < y:
#             return True
    if -100<=longi<=-35:
        y=14*longi/65+189/13
        if lat<=y:
            return True
    
    if -35<longi<-10:
        y=-14/25*longi-63/5
        if lat<y:
            return True
    
    
    if -10<=longi<=40:
        z = -9*longi/25 -43/5
        if lat<z: 
            return True
	

    if -50<= lat <=-23 and -90<=longi<=40:
        return True
    else: return False 

def getxyz(theta,phi,n):
    if n==1:
        #lat, long
        theta = np.pi/2 - theta
    x=np.sin(theta)*np.cos(phi)
    y=np.sin(theta)*np.sin(phi)
    z=np.cos(theta)
    return x,y,z
def getnorm(vec):
    norm = np.linalg.norm(vec)
    #print (norm)
    return vec/norm

In [8]:
line0 = 'ASTROSAT' #Daksha
line1 = '1 40930U 15052A   22302.17850942  .00000772  00000-0  19439-4 0  9990'
line2 = '2 40930   5.9975   0.5159 0009292 339.1937  20.7998 14.76165238112787'
line3 = '2 40930   5.9975   0.5159 0009292 339.1937 200.7998 14.76165238112787'

# line0 = 'SWIFT'#BAT
# line1 = '1 28485U 04047A   22302.18405080  .00001301  00000-0  60140-4 0  9997'
# line2 = '2 28485  20.5572 277.8742 0011565 130.4507 229.6936 15.03911323708409'
# line0 = 'FGRST (GLAST)'#Fermi
# line1 = '1 33053U 08029A   22100.45970249  .00000511  00000-0  12538-4 0  9998'
# line2 = '2 33053  25.5819 140.4060 0012690 120.1881 239.9912 15.11069703542313'
astrosat1 = ephem.readtle(line0,line1,line2)
astrosat2 = ephem.readtle(line0,line1,line3)

sublat1 = []
sublong1 = []
sublat2 = []
sublong2 = []
for i in range(len(homotime)):
    astrosat1.compute(homotime[i])
    astrosat2.compute(homotime[i])
    sublat1.append(astrosat1.sublat)#sublat in radians
    sublong1.append(astrosat1.sublong)#sublong in radian
    sublat2.append(astrosat2.sublat)#sublat in radians
    sublong2.append(astrosat2.sublong)#sublong in radian
    


In [48]:
t = homotime
yo = []
ra = []
dec = []
for i in range(220000):
    yo.append(ecef_eci(sublat1[i],sublong1[i],t[i]))
    ra.append(yo[i][0])
    dec.append(yo[i][1])
df = pd.DataFrame(data=ra)
df['declination'] = dec
df['time'] = t
df.to_csv('/Users/jinaypatel/Desktop/Daksha/ra_dec_antiearth.csv', sep=',', encoding='utf-8')
plt.plot(t[0:100],dec[0:100])